In [1]:
import random
import csv
import geopy.distance
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Airports Data")
import pickle
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


/home/lalatoofani/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lalatoofani/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
loc = []
capital = ['Islamabad','Lahore','Karachi','Mumbai','Beijing','Tehran',
           'Kabul','Baku','Dhaka','Cairo','Baghdad',
           'Tel Aviv','Astana','Kuwait City','Tripoli','Muscat','Riyadh','Abu Dhabi','Amman',
           'Moscow','London','New York City','Istanbul','Bangkok','Geneva','Colombo']
t = len(capital)
#for city in range(t):
    #location = geolocator.geocode(capital[city])
    #loc.append((location.latitude, location.longitude))

From = 'Islamabad'
To = 'London'
forbidden_from = ['Islamabad','Lahore','Karachi']
forbidden_to = ['London']
with open('lon_lat.pkl', 'rb') as f:
    city = pickle.load(f)
    capital=city[0]
    loc = city[1]
with open('large_data.csv', mode='w') as capital_distance:
    capital_writer = csv.writer(capital_distance, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    capital_writer.writerow(["Source", "Target", "Distance","s_lon","s_lat","t_lon","t_lat"])
    for i in range(t):
        for j in range(t):
            if (i!=j):
                source = loc[i]
                target = loc[j]
                if (not((capital[i] in forbidden_from and capital[j] in forbidden_to) or (capital[i] in forbidden_to and capital[j] in forbidden_from))):
                    distance = geopy.distance.vincenty(source,target).miles
                    distance = '%.2f' %distance
                    capital_writer.writerow([capital[i],capital[j],distance,source[0],source[1],target[0],target[1]])
    

/home/lalatoofani/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.


In [12]:
with open('lon_lat.pkl', 'rb') as f:
    city = pickle.load(f)
    capital=city[0]
    loc = city[1]
G = nx.Graph()
df = pd.read_csv('large_data.csv')
From = 'Islamabad'
To= 'London'

In [13]:
city = df.Source.unique()
G.add_nodes_from(city)
data = df[['Source', 'Target', 'Distance']]
edge = list(map(tuple,data.values))

In [14]:
for source,target,distance in edge:
    G.add_edge(source, target,edge_color='g', weight=distance)

In [15]:
path = nx.dijkstra_path(G, From,To)
print(path)

['Islamabad', 'Kabul', 'London']


In [16]:
a = list(nx.all_shortest_paths(G,From,To))

In [17]:
short={}
for e in a:
    total_d=0.0
    for i in range(len(e)-1):
        j = df[(df.Source == e[i])&(df.Target == e[i+1])].index
        dist = float((df.iloc[j,]['Distance']))
        total_d = total_d+dist
    short[tuple(e)]= total_d

In [21]:
import folium
lon = []
lat = []
for i in range(len(loc)):
    lon.append(loc[i][0])
    lat.append(loc[i][1])
data = pd.DataFrame({
'lat':lat,
'lon':lon,
'name':capital
})
m = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)
for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['lon'], data.iloc[i]['lat']], popup=data.iloc[i]['name']).add_to(m)
    
color = ['red','green','blue','yellow','black','brown']

print(path)
j = df[(df.Source == "Amman")&(df.Target == "Tel Aviv")].index


for i in range(len(path)-1):
    j = df[(df.Source == path[i])&(df.Target == path[i+1])].index
    lon1 = float((df.iloc[j,]['s_lon']))
    lat1 = float((df.iloc[j,]['s_lat']))
    
    lon2 = float((df.iloc[j,]['t_lon']))
    lat2 = float((df.iloc[j,]['t_lat']))
    
    dist = float((df.iloc[j,]['Distance']))
    folium.PolyLine(locations=[[lon1,lat1], [lon2,lat2]],popup=str(dist)+"miles",color=random.choice(color)).add_to(m)
m
#m.save('map.html')

['Islamabad', 'Kabul', 'London']


In [19]:
print("All possible routes are: ")
sorted_by_value = sorted(short.items(), key=lambda kv: kv[1])
for e in sorted_by_value:
    print(e)

All possible routes are: 
(('Islamabad', 'Kabul', 'London'), 3778.01)
(('Islamabad', 'Baku', 'London'), 3827.1400000000003)
(('Islamabad', 'Moscow', 'London'), 3830.11)
(('Islamabad', 'Tehran', 'London'), 3969.19)
(('Islamabad', 'Istanbul', 'London'), 4003.79)
(('Islamabad', 'Geneva', 'London'), 4008.06)
(('Islamabad', 'Astana', 'London'), 4180.9400000000005)
(('Islamabad', 'Baghdad', 'London'), 4190.610000000001)
(('Islamabad', 'Kuwait City', 'London'), 4385.73)
(('Islamabad', 'Amman', 'London'), 4409.83)
(('Islamabad', 'Tel Aviv', 'London'), 4421.18)
(('Islamabad', 'Cairo', 'London'), 4630.96)
(('Islamabad', 'Abu Dhabi', 'London'), 4739.13)
(('Islamabad', 'Muscat', 'London'), 4746.31)
(('Islamabad', 'Riyadh', 'London'), 4773.26)
(('Islamabad', 'Tripoli', 'London'), 4857.93)
(('Islamabad', 'Mumbai', 'London'), 5490.7300000000005)
(('Islamabad', 'Dhaka', 'London'), 6234.25)
(('Islamabad', 'Colombo', 'London'), 7307.17)
(('Islamabad', 'Beijing', 'London'), 7497.14)
(('Islamabad', 'Bangk